<a href="https://colab.research.google.com/github/arkapriyathecoderinprogress/InflationImpactAnalysis/blob/main/InflationImpactAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

inflation_data = pd.read_csv('/content/Inflation_Rates_Transformed-1.csv')
exchange_rate_data = pd.read_csv('/content/USD_INR_Exchange_Rates_1980_2024.csv')

In [2]:
exchange_rate_data.head()

,Year,Average Exchange Rate (INR/USD)
0,2003,45.476087
1,2004,45.188604
2,2005,44.003753
3,2006,45.183561
4,2007,41.192248


In [3]:
inflation_data.head()

,Country,Year,Inflation Rate
0,India,1980,11.3
1,United States,1980,13.5
2,Emerging market and developing economies,1980,26.0
3,India,1981,12.7
4,United States,1981,10.4


In [4]:
# filter the inflation data for India and the United States
inflation_filtered_df = inflation_data[inflation_data['Country'].isin(['India', 'United States'])]

# pivot the inflation data to have separate columns for India and the United States inflation rates
inflation_pivot_df = inflation_filtered_df.pivot(index='Year', columns='Country', values='Inflation Rate').reset_index()

# merge the exchange rates data with the inflation data
merged_df = pd.merge(exchange_rate_data, inflation_pivot_df, on='Year')

# renaming columns
merged_df.columns = ['Year', 'Exchange Rate (INR/USD)', 'Inflation Rate (India)', 'Inflation Rate (United States)']

merged_df.head()

,Year,Exchange Rate (INR/USD),Inflation Rate (India),Inflation Rate (United States)
0,2003,45.476087,4.1,2.3
1,2004,45.188604,4.0,2.7
2,2005,44.003753,3.7,3.4
3,2006,45.183561,6.7,3.2
4,2007,41.192248,5.9,2.9


In [8]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=("Trend of Exchange Rate (INR/USD)",
                                    "Trend of Inflation Rate (India)",
                                    "Trend of Inflation Rate (United Rates)"))

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Exchange Rate (INR/USD)'],
                         mode='lines+markers',
                         marker=dict(color='blue'),
                         name='Exchange Rate (INR/USD)'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Inflation Rate (India)'],
                         mode='lines+markers',
                         marker=dict(color='orange'),
                         name='Inflation Rate (India)'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Inflation Rate (United States)'],
                         mode='lines+markers',
                         marker=dict(color='green'),
                         name='Inflation Rate (United States)'),
              row=3, col=1)

fig.update_layout(height=800,
                  width=900,
                  showlegend=False,
                  title_text="Trends of Exchange Rate and Inflation Rates",
                  xaxis3_title="Year",
                  template='plotly_white')

fig.update_yaxes(title_text="Exchange Rate (INR/USD)", row=1, col=1)
fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=1)
fig.update_yaxes(title_text="Inflation Rate (%)", row=3, col=1)

fig.show()

In [9]:
correlation_matrix = merged_df[['Exchange Rate (INR/USD)',
                                'Inflation Rate (India)',
                                'Inflation Rate (United States)']].corr()

correlation_matrix

,Exchange Rate (INR/USD),Inflation Rate (India),Inflation Rate (United States)
Exchange Rate (INR/USD),1.000000,-0.337748,0.235321
Inflation Rate (India),-0.337748,1.000000,-0.120503
Inflation Rate (United States),0.235321,-0.120503,1.000000


In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Exchange Rate (INR/USD)'],
                         mode='lines+markers',
                         name='Exchange Rate (INR/USD)',
                         line=dict(color='blue')))

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Inflation Rate (India)'],
                         mode='lines+markers',
                         name='Inflation Rate (India)',
                         line=dict(color='orange')))

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Inflation Rate (United States)'],
                         mode='lines+markers',
                         name='Inflation Rate (United States)',
                         line=dict(color='green')))

fig.update_layout(title='Comparative Analysis: Exchange Rate vs Inflation Rates (India & US)',
                  xaxis_title='Year',
                  yaxis_title='Value',
                  legend_title_text='Indicators',
                  template='plotly_white',
                  height=600,
                  width=1000)

fig.show()

In [11]:
initial_exchange_rate = merged_df['Exchange Rate (INR/USD)'].iloc[0]

# calculate expected exchange rate based on PPP
merged_df['Expected Exchange Rate (PPP)'] = initial_exchange_rate * (
    (1 + merged_df['Inflation Rate (India)'] / 100) / (1 + merged_df['Inflation Rate (United States)'] / 100)).cumprod()

fig = go.Figure()

# plot actual exchange rate
fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Exchange Rate (INR/USD)'],
                         mode='lines+markers',
                         name='Actual Exchange Rate (INR/USD)',
                         line=dict(color='blue')))

# plot PPP-based expected exchange rate
fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Expected Exchange Rate (PPP)'],
                         mode='lines+markers',
                         name='Expected Exchange Rate (PPP)',
                         line=dict(color='orange', dash='dash')))

fig.update_layout(title='Actual vs. Expected Exchange Rate (PPP)',
                  xaxis_title='Year',
                  yaxis_title='Exchange Rate (INR/USD)',
                  legend_title_text='Exchange Rates',
                  template='plotly_white',
                  height=600,
                  width=1000)

fig.show()